In [1]:
import pyodbc
import pandas as pd
from pathlib import Path
import glob
import datetime
import re
import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math


In [2]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')

In [3]:
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

In [4]:
FD_path = Path( home, "desktop" , 'FD_all.xlsx')
shortage_path = Path( home, "desktop" , 'Shortage_all.xlsx')
PNbasedDetail_path = Path( home, "desktop" , 'FD_all.xlsx')

In [6]:
FD = pd.read_excel( FD_path, sheet_name='Sheet1' )
shortage = pd.read_excel( shortage_path, sheet_name='Sheet1' )
PNbasedDetail = pd.read_excel( PNbasedDetail_path, sheet_name='Sheet1' )

In [8]:
shortage

,ODM,Item,Commodity,FV,Platform,HP_PN,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside,ReportDate,last FD date,BuyerName
0,CEI,POWER98,POWERCORDS,PWC C5 1.0m stkr Conventional UK,Potter 1C18/Weasley 1C18/Potter 2C18/Maldives ...,"213351-208,213351-708,213351-108,213351-408",1042,0,0,1042,0,0,2022-12-19,2023-01-17,ChaoAngie(GPS)
1,CQQCI,2.0,LCD,11.6HD AG LED SVA EDP SLIM 250N\n11.6HD AG LED...,Ozu/Ibis/Budgie/Hitchcock,"M52883-JJ1,M52883-LJ1,M52883-RJ1,M52883-3J1,M5...",30059,0,0,30059,0,0,2022-12-19,2022-12-22,WangYu-Ting
2,CQQCI,4.0,LCD,16.1FHD AG LED UWVA EDP1.3 FLAT 300N sRGB100% ...,Taffyta22C1,"M41862-LQ2,M41862-LQ3,M41862-JQ2",0,0,0,0,0,0,2022-12-19,2022-12-19,WangYu-Ting
3,CQIEC,4.0,LCD,13.3FHD AG LED UWVA EDP SLIM-Bent 250N TOP NBZ...,Corvette,"L75254-331,L75254-L31,L75254-J32",6093,0,0,6093,0,0,2022-12-19,2023-01-22,LiLori
4,CQIEC,3.0,LCD,14(16:10) WUXGA AG LED UWVA EDP1.2 SLIM-Bent 2...,Warpath,M62321-L91,3202,0,0,3202,0,0,2022-12-19,2023-01-05,WangYu-Ting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,CQQCI,21,GCHIPSET,GN20-E3,Starmade/Valkyrie/Kerbal/Ralph\n/Cybug/Calhoun...,M12835-001,0,0,4175,4175,0,0,2022-12-19,2023-02-06,Jennifer
207,CEI,WLN31,WLAN,RT CAVA,Herschel 6U/Anders 21C1/Aldrin21C2,M34023-001,4237,0,0,4237,0,0,2022-12-19,2022-12-20,PANJESSE
208,CQQCI,26,GCHIPSET,GN21-X9,Cybug 23C1,N29054-001,0,0,809,809,0,0,2022-12-19,2023-01-17,Jennifer
209,CEI,GPU85,GCHIPSET,GN20-E7,Modena,M33431-001,0,0,100,100,0,100,2022-12-19,2050-01-01,Jennifer


In [ ]:
for index, row in FD.iterrows():
    f_ODM = row['ODM']
    f_Item = row['Item']
    f_Commodity = row['Commodity']
    f_FV = row['FV']
    f_Platform = row['Platform']
    f_HP_PN = row['HP_PN']
    f_Supplier = row['Supplier']
    f_PN = row['HP PN']
    f_ReportDate = row['ReportDate']
    f_FDdate = row['FDdate']
    f_FDQty = row['FDQty']
    f_BuyerName = row['BuyerName']

    cursor.execute(f"INSERT INTO CSI.GPS.GPS_tbl_ops_fd_temp ( ODM,Item,Commodity,FV,Platform,Supplier,[HP PN],FDdate,FDQty,Reportdate,BuyerName )\
                    VALUES('{f_ODM}','{f_Item}','{f_Commodity}','{f_FV}','{f_Platform}','{f_Supplier}','{f_PN}','{f_FDdate}','{f_FDQty}','{f_ReportDate}','{f_BuyerName}')")
conn.commit()


In [ ]:
for index, row in shortage.iterrows():
    s_ODM = row['ODM']
    s_Item = row['Item']
    s_Commodity = row['Commodity']
    s_FV = row['FV']
    s_Platform = row['Platform']
    s_P1 = row['P1']
    s_P2 = row['Net P2']
    s_P3 = row['Net P3']
    s_Total = row['Total Shortage Qty']
    s_BT = row['BT shortage']
    s_working = row['Working on upside']
    s_ReportDate = row['ReportDate']
    s_FDdate = row['last FD date']
    s_BuyerName = row['BuyerName']


    cursor.execute(f"INSERT INTO CSI.GPS.GPS_tbl_ops_fd_temp ( ODM,Item,Commodity,FV,Platform,P1,[Net P2],[Net P3],[Total Shortage Qty],[BT shortage],[Working on upside],[ReportDate],[last FD date],BuyerName )\
                    VALUES('{s_ODM}','{s_Item}','{s_Commodity}','{s_FV}','{s_Platform}','{s_P1}','{s_P2}','{s_P3}','{s_Total}','{s_BT}','{s_working}','{s_ReportDate}','{s_FDdate}','{s_BuyerName}')")
conn.commit()


In [ ]:
for index, row in PNbasedDetail.iterrows():
    ODM = row['ODM']
    Item = row['Item']
    Commodity = row['Commodity']
    FV = row['FV']
    Platform = row['Platform']
    HP_PN = row['HP_PN']
    Supplier = row['Supplier']
    PN = row['HP PN']
    ReportDate = row['ReportDate']
    FDdate = row['FDdate']
    FDQty = row['FDQty']
    BuyerName = row['BuyerName']

    cursor.execute(f"INSERT INTO CSI.GPS.GPS_tbl_ops_fd_temp ( ODM,Item,Commodity,FV,Platform,Supplier,[HP PN],FDdate,FDQty,Reportdate,BuyerName )\
                    VALUES('{ODM}','{Item}','{Commodity}','{FV}','{Platform}','{Supplier}','{PN}','{FDdate}','{FDQty}','{ReportDate}','{BuyerName}')")
conn.commit()
conn.close()